In [ ]:
from fastavro import parse_schema


schema = {
    "doc": "Spotify Wrapped Data Feed - User Interactions",
    "name": "UserInteraction",
    "namespace": "com.spotify.wrapped",
    "type": "record",
    "fields": [
        {"name": "id", "type": "long"},
        {"name": "timestamp", "type": [
            "string", "null"], "logicalType": "timestamp-millis"},
        {"name": "action", "type": ["string", "null"]},
        {"name": "song_id", "type": ["long", "null"]},
        {"name": "user_id", "type": ["long", "null"]}
    ]
}


parsed_schema = parse_schema(schema)

parsed_schema